### CH7. Function decorators and closures

* **Function decorators?**  
help "mark"

### Decorators 101

In [8]:
def deco(func):
    def inner():
        print('running inner()')
    return inner


@deco
def target():
    print('running target()')
    
target() #Invoking the decorated target actually runs inner.

running inner()


In [9]:
target #reference to inner

<function __main__.deco.<locals>.inner()>

### When Python executes decorators

* run right after the decorated function is defined

In [10]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func


@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3(): # not decorated
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__=='__main__':
    main() 

running register(<function f1 at 0x0000028FF42A53A8>)
running register(<function f2 at 0x0000028FF42A5168>)
running main()
registry -> [<function f1 at 0x0000028FF42A53A8>, <function f2 at 0x0000028FF42A5168>]
running f1()
running f2()
running f3()


 **NOTE**

- register runs (twice) before any other function in the module.  
- after the module is loaded, the registry holds references to the two decorated functions: f1 and f2.  
- Also, all is only executed by main!!
- function decorators are executed as soon as the module is imported but they only run being invoked. by ex7-2

### Decorator-enhanced Strategy pattern

In [14]:
promos = [] # list for promotion decorators

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total()* .1
        return discount
    
@promotion
def large_order(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distince_items) >= 10:
        return order.total()*.07
    return 0

def best_promo(order):
    """Select best discount available"""
    return max(promo(order) for promo in promos)

**Advantages**  
-  don’t have to use special names  
-  easy to temporarily disable a promotion: just comment out the decorator.  
-  defined in other modules, anywhere in the system

### Variable scope rules

In [16]:
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

In [18]:
f3(3)
b

3
9


9

In [20]:
b=30
f3(3)
b

3
30


9

### Closures

**NOTE**  
-  retains the bindings of the free variables that exist when the function is defined
-  the only situation in which a function may need to deal with external variables
that are non-global is when it is nested in another function.


### The nonlocal declaration

In [21]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count
    
    return averager

### Implementing a simple decorator

In [31]:
#  output the running time of functions.
import time

def clock(func):
    def clocked(*args): # Define inner function clocked to accept any number of positional arguments
        t0 = time.perf_counter()
        result = func(*args) # This line only works because the closure for clocked encompasses the func free variable.
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' %(elapsed, name, arg_str, result))
        return result
    return clocked #Return the inner function to replace the decorated function.

# other ex exists clock_demo.py

**How it works**
1. records the initial time t0  
2. calls the original factorial, saving the result  
3. computes the elapsed time  
4. formats and prints the collected data  
5. returns the result saved in step 2

### Memoization with functools.lru_cache

In [34]:
import functools

from clockdeco import clock

@functools.lru_cache()
@clock
def fibonacci(n):
    if n<2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__ =='__main__':
    print(fibonacci(6))

ModuleNotFoundError: No module named 'clockdeco'

### Generic functions with single dispatch

In [36]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch #@singledispatch marks the base function which handles the object type.
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str) #Each specialized function is decorated with @«base_function».register(«type»).

def _(text): # The name of the specialized functions is irrelevant; _ is a good choice to make this clear.
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral) #For each additional type to receive special treatment,register a new function.numbers.Integral is a virtual superclass of int (see below).
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple) # You can stack several register decorators to support different types with the samefunction.
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

**NOTE**  
The new functools.singledispatch decorator in Python 3.4 allows each module to
contribute to the overall solution, and lets you easily provide a specialized function even
for classes that you can’t edit. 

### A parametrized registration decorator

In [37]:
registry = set()

def register(active=True):
    def decorate(func):
        print('running register(active=%s)->decorate(%s)' %(active, func))
        if active: #Register func only if the active argument (retrieved from the closure) is True.
            registry.add(func)
        else:
            registry.discard(func)
        
        return func
    return decorate

@register(active=False)
def f1():
    print('running f1()')
    
@register() # If no parameters are passed, register must still be called as a function — @register() — to return the actual decorator, decorate. 
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')

running register(active=False)->decorate(<function f1 at 0x0000028FF42EB168>)
running register(active=True)->decorate(<function f2 at 0x0000028FF42CBF78>)


### The parametrized clock decorator

In [39]:
# Module clockdeco_param.py: the parametrized clock decorator
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT): #clock is our parametrized decorator factory
    def decorate(func): #decorate is the actual decorator.
        def clocked(*_args): #clocked wraps the decorated function.
            t0 = time.time()
            _result = func(*_args) #_result is the actual result of the decorated function.
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args) #_args holds the actual arguments of clocked, while args is str used for display
            result = repr(_result) #result is the str representation of _result, for display..
            print(fmt.format(**locals()))    
            return _result 
        return clocked # decorate returns clocked
    return decorate # clock returns decorate.

if __name__=='__main__':
    
    @clock() 
    def snooze(seconds):
        time.sleep(seconds)
    
    for i in range(3):
        snooze(.123)

[0.12366748s] snooze(0.123) -> None
[0.12366796s] snooze(0.123) -> None
[0.12366962s] snooze(0.123) -> None


### CH8 . Object references, mutability and recycling

In [40]:
class Gizmo:
    def __init__(self):
        print('Gizmo id: %d' %id(self))

In [41]:
x = Gizmo()
y = Gizmo()*10

Gizmo id: 2817300193672
Gizmo id: 2817300196232


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

### Identity, equality and aliases

In [42]:
charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles # lewis is an alias for charles.
lewis is charles

True

In [44]:
id(charles), id(lewis) # The is operator and the id function confirm it
lewis['balance'] = 950 # Adding an item to lewis is the same as adding an item to charles
charles

{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

### Choosing between == and is

The == operator compares the values of objects (the data they hold), while is compares
their identities.

### The relative immutability of tuples

 behind the riddle “A += assignment puzzler” on
page 40. It’s also the reason why some tuples are unhashable

### Copies are shallow by default

**NOTE**
- The easiest way to copy a list (or most built-in mutable collections) is to use the builtin constructor for the type itself
-  shallow copies are easy to make, but they may or may not
be what you want.

### Deep and shallow copies of arbitrary objects

- you need deep copies when duplicating that do not share references of embedded objects
-  making deep copies is not a simple matter in the general case. especially cyclic references!!

In [45]:
class Bus:
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
            
    def pick(slef, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

In [46]:
import copy
bus1 = Bus(['Alice', 'Bill', ' Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)

### Function parameters as references

### Mutable types as parameter defaults: bad idea

-  you
should avoid mutable objects as default values for parameters.

In [49]:
class HauntedBus:
    """A bus model haunted by ghost passengers"""
    def __init__(self, passengers=[]):
        self.passengers = passengers
    def pick(self, name):
        self.passengers.append(name)
    def drop(self, name):
        self.passengers.remove(name)

In [51]:
bus1 = HauntedBus(['Alice', 'Bill'])
bus1.passengers
bus1.pick('Charlie')
bus1.drop('Alice')
bus1.passengers

In [54]:
bus2 = HauntedBus()
bus2.pick('Carrie')
bus1.passengers # But bus1.passengers is a distinct list

['Bill', 'Charlie']

### Defensive programming with mutable parameters

In [47]:
class TwilightBus:
    """A bus model haunted by ghost passengers"""
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
        
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

**Differences**  
 our internal handling of the passenger list will not affect the argument used to
initialize the bus.

### del and garbage collection

In [60]:
import weakref
s1 = {1, 2, 3}
s2 = s1

def bye(): #This function must not be a bound method the object about to be destroyed or otherwise hold a reference to it.
    print('Gone with the wind...')
ender = weakref.finalize(s1, bye) # Register the bye callback on the object referred by s1.
ender.alive # The .alive attribute is True before the finalize object is called
del s1
ender.alive
s2 = 'spam' #Rebinding the last reference, s2, makes {1, 2, 3} unreachable. It is destroyed, the bye callback is invoked and ender.alive becomes False.

ender.alive

Gone with the wind...
Gone with the wind...


False

### Weak references

- is useful to have a reference to an object that does not keep it around longer than necessary
- The object that is the
target of a reference is called the referent. 
- Therefore, we say that a weak reference does
not prevent the referent from being garbage collected.


In [62]:
import weakref
a_set = {0, 1}
wref = weakref.ref(a_set) # The wref weak reference object is created and inspected in the next line
wref

wref()

a_set = {2, 3, 4}
wref() # Calling wref() still returns {0, 1}.


wref() is None # Because the {0, 1} object is now gone, this last call to wref() returns None

True

### Limitations of weak references

- Not every Python object may be the target, or referent, of a weak reference. Basic list
and dict instances may not be referents, but a plain subclass of either can solve this
problem easily
- They are the result of internal optimizations, some of which
are discussed in the following (highly optional) section.
